In [113]:
import pathlib
import itertools

import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt

from calcium_bflow_analysis.single_fov_analysis import filter_da
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
foldername = pathlib.Path('/data/Amit_QNAP/Calcium_FXS/')
fname_glob = '*.nc'
full_fnames = next(foldername.glob(fname))
full_fnames

PosixPath('/data/Amit_QNAP/Calcium_FXS/data_of_day_-1.nc')

In [19]:
data = xr.open_dataset(full_fname)
data

<xarray.Dataset>
Dimensions:      (epoch: 12, fname: 21, neuron: 85, time: 9000)
Coordinates:
  * time         (time) float64 0.0 0.03329 0.06658 ... 299.5 299.5 299.6
  * neuron       (neuron) int32 0 1 2 3 4 5 6 7 8 ... 76 77 78 79 80 81 82 83 84
  * fname        (fname) object 'FXS_609_X10_FOV1_mag3_20181009_00001' ... 'WT_674_X10_FOV4_mag3_20181009_00004'
    mouse_id     (fname) object ...
    condition    (fname) object ...
    fov          (fname) int32 ...
  * epoch        (epoch) object 'run_stim' 'run_juxta' ... 'spont' 'all'
    day          int32 ...
Data variables:
    dff          (fname, neuron, time) float64 ...
    epoch_times  (fname, epoch, time) bool ...
Attributes:
    fps:          30.04
    stim_window:  2.0

In [98]:
data.sel(epoch='spont').where(data.mouse_id == '609', drop=True)

<xarray.Dataset>
Dimensions:      (fname: 4, neuron: 85, time: 9000)
Coordinates:
  * time         (time) float64 0.0 0.03329 0.06658 ... 299.5 299.5 299.6
  * neuron       (neuron) int32 0 1 2 3 4 5 6 7 8 ... 76 77 78 79 80 81 82 83 84
  * fname        (fname) object 'FXS_609_X10_FOV1_mag3_20181009_00001' ... 'FXS_609_X10_FOV4_mag3_20181009_00004'
    mouse_id     (fname) object '609' '609' '609' '609'
    condition    (fname) object 'FXS' 'FXS' 'FXS' 'FXS'
    fov          (fname) int32 1 2 3 4
    epoch        <U5 'spont'
    day          int32 1
Data variables:
    dff          (fname, neuron, time) float64 1.186 0.6939 0.4073 ... nan nan
    epoch_times  (fname, time) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
Attributes:
    fps:          30.04
    stim_window:  2.0

In [26]:
spont_wfa = filter_da(data, 'spont', 'FXS')
spont_wt = filter_da(data, 'spont', 'WT')


In [85]:
data.dff.where(data.mouse_id == '609', drop=True)

<xarray.DataArray 'dff' (fname: 4, neuron: 85, time: 9000)>
array([[[ 1.18568913,  0.69392571,  0.4073198 , ...,  0.27191847,
          0.16624921,  0.37564509],
        [ 0.40115553,  0.31092911,  0.90714348, ..., -0.05824346,
          0.28990059,  0.26180533],
        [-0.04738316,  0.61461214, -0.13916785, ..., -0.13261082,
          0.32732599, -0.05911888],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[-0.04472624,  0.03888464, -0.20769215, ...,  0.29842064,
          0.15372955,  0.29343916],
        [-0.02641277, -0.04887139,  0.81944638, ...,  0.36091871,
          0.425631  ,  1.3720757 ],
        [ 0.01328615,  0.11102065,  0.92790892, ...,  0.1866958 ,
         -0.12060112, -0.10713656],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[ 0.07436235,  0.57295431,  0.18964091, ...,  2.66382823,
          1.13947861,  1.05202333],
        [-0.20186556,  0.62382751,  0.45405547, ...,  0.67087041,
          0.34492817,  0.39248308],
        [-0.17597222, -0.00406243,  0.27054891, ..., -0.03157993,
          0.14931379,  0.09562847],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[ 0.37081214, -0.13279844,  0.44918426, ..., -0.47072121,
          0.01208895,  0.00649542],
        [ 0.00461759,  0.16079885,  1.48338696, ...,  0.72815947,
          0.40002182, -0.07261287],
        [ 0.36929403,  0.81601098,  0.17855802, ...,  0.13730565,
          0.63021135,  0.12176126],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]]])
Coordinates:
  * time       (time) float64 0.0 0.03329 0.06658 0.09987 ... 299.5 299.5 299.6
  * neuron     (neuron) int32 0 1 2 3 4 5 6 7 8 9 ... 76 77 78 79 80 81 82 83 84
  * fname      (fname) object 'FXS_609_X10_FOV1_mag3_20181009_00001' ... 'FXS_609_X10_FOV4_mag3_20181009_00004'
    mouse_id   (fname) object '609' '609' '609' '609'
    condition  (fname) object 'FXS' 'FXS' 'FXS' 'FXS'
    fov        (fname) int32 1 2 3 4
    day        int32 1

In [70]:
data.sel(epoch="spont").where(data.condition == 'FXS', drop=True).where(data.mouse_id == '609', drop=True).dff.shape

(4, 85, 9000)

In [84]:
fxs_wt = {'FXS': {'spont': [], 'stim': [], 'all': []}, 'WT': {'spont': [], 'stim': [], 'all': []}}
epoch = ('spont', 'stim', 'all')
genotype = ('FXS', 'WT')
# tuple the generator to make it last after first iteration
pairs = tuple(itertools.product(epoch, genotype))

for mouse_id, ds in data.groupby('mouse_id'):
    for pair in pairs:
        dff = filter_da(ds, *pair)
        print(f'epoch: {pair[0]}, geno: {pair[1]}, id: {mouse_id} dff.shape: {dff.shape}')
        print(dff)


epoch: spont, geno: FXS, id: 609 dff.shape: (54, 9000)
[[ 1.18568913  0.69392571  0.4073198  ...         nan         nan
          nan]
 [ 0.40115553  0.31092911  0.90714348 ...         nan         nan
          nan]
 [-0.04738316  0.61461214 -0.13916785 ...         nan         nan
          nan]
 ...
 [-0.11152138 -0.25689408  0.17898168 ...         nan         nan
          nan]
 [ 0.63744407  0.22782354 -0.27461757 ...         nan         nan
          nan]
 [-0.0756089  -0.17540386  0.0470988  ...         nan         nan
          nan]]
epoch: spont, geno: WT, id: 609 dff.shape: (54, 9000)
[[ 1.18568913  0.69392571  0.4073198  ...         nan         nan
          nan]
 [ 0.40115553  0.31092911  0.90714348 ...         nan         nan
          nan]
 [-0.04738316  0.61461214 -0.13916785 ...         nan         nan
          nan]
 ...
 [-0.11152138 -0.25689408  0.17898168 ...         nan         nan
          nan]
 [ 0.63744407  0.22782354 -0.27461757 ...         nan         nan
    

epoch: spont, geno: FXS, id: 640 dff.shape: (65, 9000)
[[ 0.39405737 -0.68427553 -0.81978323 ...         nan         nan
          nan]
 [-0.45474074 -0.40987501  0.07347317 ...         nan         nan
          nan]
 [ 0.06675067 -0.04078707 -0.17293675 ...         nan         nan
          nan]
 ...
 [ 0.26565421  0.73063052  0.29361725 ...         nan         nan
          nan]
 [ 0.30591191 -0.03287353  0.46413046 ...         nan         nan
          nan]
 [-0.45275513  0.18654628 -0.38007112 ...         nan         nan
          nan]]
epoch: spont, geno: WT, id: 640 dff.shape: (65, 9000)
[[ 0.39405737 -0.68427553 -0.81978323 ...         nan         nan
          nan]
 [-0.45474074 -0.40987501  0.07347317 ...         nan         nan
          nan]
 [ 0.06675067 -0.04078707 -0.17293675 ...         nan         nan
          nan]
 ...
 [ 0.26565421  0.73063052  0.29361725 ...         nan         nan
          nan]
 [ 0.30591191 -0.03287353  0.46413046 ...         nan         nan
    

In [136]:
s = data.where(data.mouse_id == '609', drop=True)
s['epoch_times'] = s['epoch_times'].astype(bool)
filter_da(s, 'spont', 'WT')

array([], shape=(0, 9000), dtype=float64)

In [133]:
epoch_data = ds.sel(epoch='spont')
temp = epoch_data.where(epoch_data.condition == 'condition')
temp['epoch_times'] = temp['epoch_times'].astype(bool)
epoch_data = temp

In [134]:
epoch_data

<xarray.Dataset>
Dimensions:      (fname: 3, neuron: 85, time: 9000)
Coordinates:
  * time         (time) float64 0.0 0.03329 0.06658 ... 299.5 299.5 299.6
  * neuron       (neuron) int32 0 1 2 3 4 5 6 7 8 ... 76 77 78 79 80 81 82 83 84
  * fname        (fname) object 'WT_674_X10_FOV2_mag3_20181009_00002' ... 'WT_674_X10_FOV4_mag3_20181009_00004'
    mouse_id     (fname) object '674' '674' '674'
    condition    (fname) object 'WT' 'WT' 'WT'
    fov          (fname) int32 2 3 4
    epoch        <U5 'spont'
    day          int32 1
Data variables:
    dff          (fname, neuron, time) float64 nan nan nan nan ... nan nan nan
    epoch_times  (fname, time) bool True True True True ... True True True True
Attributes:
    fps:          30.04
    stim_window:  2.0